<a href="https://colab.research.google.com/github/farrukhpasha/DataScience-Public/blob/main/Quran_Haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Enable the GPU Support and make sure the GPU is enabled.

In [4]:
!nvidia-smi

Tue Feb  2 19:11:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install the latest master of Haystack


In [5]:
# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4
!pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-gsg0hrau
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-gsg0hrau
  Created wheel for farm-haystack: filename=farm_haystack-0.7.0-cp36-none-any.whl size=127436 sha256=a5c343c74b885e8d3fbf1addd321d1bee51ca59a805ed7b8d1e41e029d33358f
  Stored in directory: /tmp/pip-ephem-wheel-cache-8__dmrjm/wheels/ab/41/a4/4fbf362de283352078ecb6705c08b6525347aaea2eead2a60c
Successfully built farm-haystack
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [6]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

02/02/2021 19:11:42 - INFO - faiss -   Loading faiss with AVX2 support.
02/02/2021 19:11:42 - INFO - faiss -   Loading faiss.


In [7]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [8]:
# Let's first get some files that we want to use
doc_dir = "sample_data/Quran"
#s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
#fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/010.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/012.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/076.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/013.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/003.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/101.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/107.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/008.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/090.txt
02/02/2021 19:11:43 - INFO - haystack.preprocessor.utils -   Converting sample_data/Quran/067.txt
02/02/2021 19:11:43 

In [9]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

02/02/2021 19:12:01 - INFO - haystack.document_store.faiss -   Updating embeddings for 114 docs...


In [10]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

02/02/2021 19:12:03 - INFO - farm.utils -   Using device: CUDA 
02/02/2021 19:12:03 - INFO - farm.utils -   Number of GPUs: 1
02/02/2021 19:12:03 - INFO - farm.utils -   Distributed Training: False
02/02/2021 19:12:03 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
02/02/2021 19:12:12 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='prediction_heads' was already logged with value='TextSimilarityHead' for run ID='f72987091e944596a6cd3710496e9501'. Attempted logging new value 'QuestionAnsweringHead'.
02/02/2021 19:12:14 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='proce

In [11]:
finder = Finder(reader, retriever)

02/02/2021 19:12:14 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [12]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
#prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_retriever=10, top_k_reader=5)

#prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_retriever=10, top_k_reader=5)

prediction = finder.get_answers(question="is Honey bee mentioned?", top_k_retriever=10, top_k_reader=5)
print_answers(prediction, details="minimal")

02/02/2021 19:12:14 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
02/02/2021 19:12:14 - INFO - haystack.finder -   Got 10 candidates from retriever
02/02/2021 19:12:14 - INFO - haystack.finder -   Reader is looking for detailed answer in 76349 chars ...


[   {   'answer': 'All praise belongs to Allah',
        'context': 'h he gives secretly and openly. Are they equal in '
                   'comparison? All praise belongs to Allah, but most of them '
                   'do not know.\n'
                   '76. And Allah cites the examp'},
    {   'answer': 'Abraham',
        'context': 'gdoers."\n'
                   '60. They said, "We heard a youth mentioning them. He is '
                   'called Abraham."\n'
                   '61. They said, "Bring him before the eyes of the people, '
                   'so that the'},
    {   'answer': 'a youth',
        'context': ' gods? He is certainly one of the wrongdoers."\n'
                   '60. They said, "We heard a youth mentioning them. He is '
                   'called Abraham."\n'
                   '61. They said, "Bring him befo'},
    {   'answer': 'Moses',
        'context': ' gave them a noble reputation of truth.\n'
                   '51. And mention in the Scripture Mos

In [13]:
#print_answers(prediction, details="minimal")